In [1]:
%load_ext lab_black

Step: Load data

In [2]:
import pandas as pd
import os
from pyprojroot import here
import numpy as np
from skimpy import clean_columns
from data_cleaning.fun_hot_encode_limit import fun_hot_encode_limit

path_data = here("./data")
os.chdir(path_data)
data_iu = pd.read_csv("iu_data.csv")
data_iu = clean_columns(data_iu)

17 column names have been cleaned

Step: Drop year

In [4]:
data_iu["season"] = data_iu["season"].apply(lambda x: x.split("-")[0])

data_iu["season"] = pd.to_datetime(data_iu["season"], format="%Y")

data_iu_update = data_iu.query("season >= '1949-01-01'").drop(columns=["coach_es"])

Step: Create the outcome variable

In [5]:
data_ncaa_tournament = pd.get_dummies(data_iu_update["ncaa_tournament"], dummy_na=True)
data_ncaa_tournament = clean_columns(data_ncaa_tournament)
data_ncaa_tournament = data_ncaa_tournament.rename(columns={"nan": "tournament_make"})
data_ncaa_tournament = np.where(data_ncaa_tournament["tournament_make"] == 0, 1, 0)

7 column names have been cleaned

Step: Drop date and variables

In [6]:
data_iu_encoded = pd.concat(
    [
        data_iu_update.drop(columns="ncaa_tournament"),
        pd.DataFrame(data_ncaa_tournament),
    ],
    axis=1,
).rename(columns={0: "tournament_make"})

Step: Remove *

In [7]:
columns_to_remove_asterisk = data_iu_encoded.columns
columns_to_remove_asterisk

Index(['season', 'w', 'l', 'w_l_%', 'w_c', 'l_c', 'w_l_%_c', 'srs', 'sos',
       'ps_g', 'pa_g', 'ap_pre', 'ap_high', 'ap_final', 'seed',
       'tournament_make'],
      dtype='object')

In [8]:
for col in columns_to_remove_asterisk:
    data_iu_encoded[col] = data_iu_encoded[col].replace("\*", "", regex=True)

In [9]:
data_iu_encoded[["w", "l", "w_c", "l_c"]] = data_iu_encoded[
    ["w", "l", "w_c", "l_c"]
].astype(int)

Step: Remove deterministic variables

In [12]:
columns_drop = ["season", "w_l_%", "w_l_%_c", "srs", "ap_high", "seed"]
data_iu_encoded = data_iu_encoded.drop(columns=columns_drop)

Step: Save data

In [13]:
data_iu_encoded.to_parquet("data_iu_analysis.parquet")